In [7]:
%%time
import duckdb 
con = duckdb.connect()
df =con.execute('''
SELECT     l_returnflag,  l_linestatus,    SUM(l_quantity) AS sum_qty,    SUM(l_extendedprice) AS sum_base_price,   SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,    COUNT(*) AS count_order
FROM   'lineitem.parquet'
where L_shipdate='1994-01-01'
GROUP BY    l_returnflag,    l_linestatus ORDER BY    l_returnflag,    l_linestatus;
''').df()
df

CPU times: total: 22.8 s
Wall time: 4.02 s


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,637489.0,9.552864e+08,9.074077e+08,9.434766e+08,25.381789,38034.973885,0.050131,25116
1,R,F,636976.0,9.550224e+08,9.074514e+08,9.436636e+08,25.474964,38194.785428,0.049928,25004


In [8]:
%%time
import duckdb 
con = duckdb.connect()
df =con.execute('''
SELECT     l_returnflag,  l_linestatus,    SUM(l_quantity) AS sum_qty,    SUM(l_extendedprice) AS sum_base_price,   SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,    COUNT(*) AS count_order
FROM   'lineitem_sorted.parquet'
where L_shipdate = '1994-01-01'
GROUP BY    l_returnflag,    l_linestatus ORDER BY    l_returnflag,    l_linestatus;
''').df()
df

CPU times: total: 281 ms
Wall time: 241 ms


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,637489.0,9.552864e+08,9.074077e+08,9.434766e+08,25.381789,38034.973885,0.050131,25116
1,R,F,636976.0,9.550224e+08,9.074514e+08,9.436636e+08,25.474964,38194.785428,0.049928,25004


You can Query Parquet metadata using DuckDB

In [9]:
import duckdb
con = duckdb.connect()
metadata = con.execute("SELECT * FROM parquet_metadata('lineitem.parquet')").df()
metadata

,file_name,row_group_id,row_group_num_rows,row_group_num_columns,row_group_bytes,column_id,file_offset,num_values,path_in_schema,type,...,stats_distinct_count,stats_min_value,stats_max_value,compression,encodings,index_page_offset,dictionary_page_offset,data_page_offset,total_compressed_size,total_uncompressed_size
0,lineitem.parquet,0,100352,16,0,0,0,100352,l_orderkey,INT32,...,NaN,1,99941,ZSTD,PLAIN,0,0,4,37113,0
1,lineitem.parquet,0,100352,16,0,1,0,100352,l_partkey,INT32,...,NaN,33,3999993,ZSTD,PLAIN,0,0,37117,332970,0
2,lineitem.parquet,0,100352,16,0,2,0,100352,l_suppkey,INT32,...,NaN,2,199997,ZSTD,PLAIN,0,0,370087,283134,0
3,lineitem.parquet,0,100352,16,0,3,0,100352,l_linenumber,INT32,...,NaN,1,7,ZSTD,PLAIN,0,0,653221,37022,0
4,lineitem.parquet,0,100352,16,0,4,0,100352,l_quantity,INT32,...,NaN,1,50,ZSTD,PLAIN,0,0,690243,103762,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19131,lineitem.parquet,1195,73968,16,0,11,0,73968,l_commitdate,INT32,...,NaN,1992-02-03,1998-10-31,ZSTD,PLAIN,0,0,3333084143,129726,0
19132,lineitem.parquet,1195,73968,16,0,12,0,73968,l_receiptdate,INT32,...,NaN,1992-01-14,1998-12-23,ZSTD,PLAIN,0,0,3333213869,131858,0
19133,lineitem.parquet,1195,73968,16,0,13,0,73968,l_shipinstruct,BYTE_ARRAY,...,4.0,COLLECT COD,TAKE BACK RETURN,ZSTD,"PLAIN, RLE_DICTIONARY",0,3333345727,3333345792,29409,0
19134,lineitem.parquet,1195,73968,16,0,14,0,73968,l_shipmode,BYTE_ARRAY,...,7.0,AIR,TRUCK,ZSTD,"PLAIN, RLE_DICTIONARY",0,3333375136,3333375203,37825,0


Check the number of row groups that contain l_shipdate = '1994-01-01'

In [10]:
filter  = con.execute('''
                      SELECT path_in_schema,row_group_id,stats_min_value,stats_max_value  FROM metadata
                      where path_in_schema = 'l_shipdate'
                      and (stats_min_value <= '1994-01-01' and stats_max_value >= '1994-01-01')
                      ''').df()
filter

,path_in_schema,row_group_id,stats_min_value,stats_max_value
0,l_shipdate,0,1992-01-03,1998-11-29
1,l_shipdate,1,1992-01-04,1998-11-30
2,l_shipdate,2,1992-01-07,1998-11-28
3,l_shipdate,3,1992-01-03,1998-12-01
4,l_shipdate,4,1992-01-03,1998-12-01
...,...,...,...,...
1191,l_shipdate,1191,1992-01-04,1998-11-30
1192,l_shipdate,1192,1992-01-02,1998-11-30
1193,l_shipdate,1193,1992-01-02,1998-12-01
1194,l_shipdate,1194,1992-01-04,1998-11-30


For the sorted file, DuckDB needs to scan only 2 row groups, hence the better performance

In [13]:
import duckdb
con = duckdb.connect()
metadata = con.execute("SELECT * FROM parquet_metadata('lineitem_sorted.parquet')").df()
filter  = con.execute('''
                      SELECT path_in_schema,row_group_id,stats_min_value,stats_max_value  FROM metadata
                      where path_in_schema = 'l_shipdate'
                      and (stats_min_value <= '1994-01-01' and stats_max_value >= '1994-01-01')
                      ''').df()
filter

,path_in_schema,row_group_id,stats_min_value,stats_max_value
0,l_shipdate,332,1993-12-29,1994-01-01
1,l_shipdate,333,1994-01-01,1994-01-03
